original Input:
選擇類別 inputLabel
關鍵字排名區間 K
起始日 year_from, month_from
終止日 year_to, month_to

In [1]:
import pandas as pd
XX = pd.read_csv('NER_analysis_data/DocDTM.csv') # the DTM illustrates whether the keyword exist in the artical or not 
E = pd.read_csv('NER_analysis_data/NER_old/entityDict.csv') # keywords, label, word_label_count, freq
X = pd.read_csv('NER_analysis_data/NER_old/doc_raw_data.csv') # doc_id, linK, title, date, subject, ner_doc

In [7]:
print('DocDTM.csv\n')
print(X.head(4))
print('==========================================')
print('doc_raw_data.csv\n')
print(XX.head(4))

DocDTM.csv

   doc_id                                               link  \
0       0  https://spacenews.com/france-joins-asat-testin...   
1       1  https://spacenews.com/wide-field-of-view-missi...   
2       2  https://spacenews.com/hiber-demands-astrocast-...   
3       3  https://spacenews.com/ororatech-raises-funding...   

                                               title                 date  \
0               France joins ASAT testing moratorium  2022-11-30 00:00:00   
1  Wide-Field-of-View missile warning satellite t...  2022-11-30 00:00:00   
2  Hiber demands Astrocast cash after failed acqu...  2022-11-30 00:00:00   
3  OroraTech raises funding for additional therma...  2022-11-30 00:00:00   

  subject                                            ner_doc  
0    news  WASHINGTON — The French government said Nov. 2...  
1    news  WASHINGTON — The U.S. Space Force has received...  
2    news  TAMPA, Fla. — Dutch remote monitoring speciali...  
3    news  WASHINGTON — Orora

In [2]:
print(len(X),
len(XX))

10982 10962


In [8]:
# segment date by month
X['date'] = pd.to_datetime(X['date'])
X['date'] = pd.to_datetime(X['date'].dt.strftime('%Y-%m'))


In [9]:
# group doc_id in X by date
X = X.groupby('date')['doc_id'].apply(list).reset_index()
X

,date,doc_id
0,2017-01-01,"[7212, 7213, 7214, 7215, 7216, 7217, 7218, 721..."
1,2017-02-01,"[7114, 7115, 7116, 7117, 7118, 7119, 7120, 712..."
2,2017-03-01,"[7004, 7005, 7006, 7007, 7008, 7009, 7010, 701..."
3,2017-04-01,"[6918, 6919, 6920, 6921, 6922, 6923, 6924, 692..."
4,2017-05-01,"[6823, 6824, 6825, 6826, 6827, 6828, 6829, 683..."
...,...,...
75,2023-04-01,"[10228, 10229, 10230, 10231, 10232, 10233, 102..."
76,2023-05-01,"[10394, 10395, 10396, 10397, 10398, 10399, 104..."
77,2023-06-01,"[10553, 10554, 10555, 10556, 10557, 10558, 105..."
78,2023-07-01,"[10706, 10707, 10708, 10709, 10710, 10711, 107..."


In [10]:
# create a dataframe named freq_DTM get keywords and their corresponding label from E 
freq_DTM = E.loc[:, ['keywords', 'label']]
freq_DTM



,keywords,label
0,3D printing,term
1,5G,term
2,ABL,com
3,ABS,com
4,AEHF,satellite
...,...,...
563,synthetic aperture radar,term
564,technical,term
565,technology,term
566,time,term


In [11]:
# inspect how many months in our data
len(X)

80

In [13]:
# because the 'doc_id' in 'XX' are more than in 'X' we have to trim off redundant 'doc_id'
X['doc_id'][len(X)-1]=[doc_id for doc_id in X['doc_id'][79] if doc_id <= (len(XX)-1)]

C:\Users\user\AppData\Local\Temp\ipykernel_3816\2008695584.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['doc_id'][len(X)-1]=[doc_id for doc_id in X['doc_id'][79] if doc_id <= (len(XX)-1)]


In [17]:
# add columns by sum of the keywords show up per month 
for i in range(len(X)):
    d = X['date'][i].strftime('%Y-%m')
    freq_DTM[d] = list(XX.loc[X['doc_id'][i]].sum())

In [18]:
freq_DTM

,keywords,label,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,...,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08
0,3D printing,term,0,0,1,0,0,1,0,1,...,1,0,0,2,2,2,2,0,4,0
1,5G,term,2,0,0,0,0,10,0,0,...,11,2,2,7,4,3,11,1,4,0
2,ABL,com,0,0,0,0,0,0,0,0,...,0,0,9,0,4,0,3,4,0,0
3,ABS,com,7,2,2,2,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,AEHF,satellite,0,0,0,0,0,0,1,12,...,0,0,0,0,0,1,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,synthetic aperture radar,term,0,1,0,4,0,1,0,4,...,1,3,6,5,7,4,6,2,7,0
564,technical,term,2,2,0,0,2,0,4,2,...,5,5,3,1,0,5,2,1,0,0
565,technology,term,3,2,3,7,3,21,3,8,...,3,1,3,4,1,6,5,6,4,1
566,time,term,1,3,2,1,1,3,0,3,...,2,0,3,4,1,2,4,0,0,1


In [19]:
freq_DTM.loc[freq_DTM['keywords']=='Air Force']['2018-12']


16    132
Name: 2018-12, dtype: int64

In [20]:
# Export the DataFrame to a CSV file
csv_filename = 'freq_DTM.csv'
freq_DTM.to_csv(csv_filename, index=False)

In [67]:
freq_DTM.loc[freq_DTM['label']=='term'].loc[:, '2017-05':'2017-08']
freq_DTM0 = freq_DTM.loc[freq_DTM['label']=='term'].loc[:, '2017-05':'2017-08']
freq_DTM0['keywords'] = freq_DTM.loc[freq_DTM['label']=='term']['keywords'] 
freq_DTM0['period'] = freq_DTM0.loc[:, '2017-05':'2017-08'].sum(axis=1)
freq_DTM0

,2017-05,2017-06,2017-07,2017-08,keywords,period
0,0,1,0,1,3D printing,2
1,0,10,0,0,5G,10
6,0,0,3,10,AI,13
7,0,0,11,0,ASAT,11
32,4,0,4,3,Apollo,11
...,...,...,...,...,...,...
563,0,1,0,4,synthetic aperture radar,5
564,2,0,4,2,technical,8
565,3,21,3,8,technology,35
566,1,3,0,3,time,7


In [74]:
freq_DTM1 =freq_DTM0.loc[:,'2017-05':'2017-08']

In [78]:
freq_DTM1[freq_DTM0['keywords'] == 'GEO'][list(
            freq_DTM1.columns[:-2])].values.flatten().tolist()

[1, 17]

In [81]:
freq_DTM1.columns
for column in freq_DTM1.columns:
    print(column)

Index(['2017-05', '2017-06', '2017-07', '2017-08'], dtype='object')

In [65]:
for keyword in selected_keywords:
        y_values = freq_DTM0[freq_DTM0['keywords'] == keyword][list(
            freq_DTM0.columns[:-2])].values.flatten().tolist()
        print(y_values)

[4, 0, 4, 3]
[0, 0, 0, 1]
[0, 0, 0, 0]
[4, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 4, 0, 2]
[2, 3, 3, 0]
[10, 2, 4, 0]
[0, 1, 0, 1]
[1, 17, 6, 3]


In [44]:
for column in freq_DTM1.columns:
    col_sum = freq_DTM1[column].sum()  # Calculate the sum of the column
    freq_DTM1[column] = freq_DTM1[column] / col_sum

In [21]:
freq_DTM0 = freq_DTM0.sort_values(by='period',  ascending=False).head(50)

In [18]:
freq_DTM0.iloc[2:]

,2017-05,2017-06,2017-07,2017-08,keywords,period
6,0,0,3,10,AI,13
7,0,0,11,0,ASAT,11
32,4,0,4,3,Apollo,11
40,0,0,0,1,Artemis,1
64,0,0,0,0,Biden,0
...,...,...,...,...,...,...
563,0,1,0,4,synthetic aperture radar,5
564,2,0,4,2,technical,8
565,3,21,3,8,technology,35
566,1,3,0,3,time,7
